In [ ]:
import json
import pandas as pd
import random
import torch
import requests

In [ ]:
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer xxx"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": "Can you please let us know more details about your ",
})

In [ ]:
# define the function to make a request to the Hugging Face API

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [ ]:
# function to generate a sentence containing the given word using the API max 50 tokens

def generate_sentence(prompt, max_tokens=50):

    payload = {
        "inputs": prompt,
        "parameters": {"max_tokens": max_tokens}  # add token limit parameter
    }
    response = query(payload)

    if isinstance(response, dict) and "error" in response:
        print(f"Error from API: {response['error']}")
        return ""
    generated_text = response[0]["generated_text"]
    # remove the prompt part from the generated text
    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

### Sentence containing

In [ ]:
# Load the dataset
with open('/content/sentence_containing.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)  # for reproducibility
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

In [ ]:
# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # use the API to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence contains the word
    score = 1 if word in generated_sentence else 0

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

In [ ]:
# to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to JSON files
df.to_csv('eval_sentence_containing.csv', index=False)

   example_number                                              input  \
0               1  Составьте предложение, используя слово "зватьс...   
1               2     Напишите предложение со словом "фантастика":\n   
2               3       Напишите предложение со словом "выяснять":\n   
3               4  Придумайте предложение, в котором будет слово ...   
4               5  Придумайте предложение, в котором будет слово ...   
5               6         Напишите предложение со словом "опасно":\n   
6               7    Напишите предложение со словом "француженка":\n   
7               8   Напишите предложение со словом "обязательный":\n   
8               9  Составьте предложение, используя слово "испуга...   
9              10  Составьте предложение, используя слово "разный...   

            word word_pos_tags  \
0        зваться        [VERB]   
1     фантастика        [NOUN]   
2       выяснять        [VERB]   
3  востребование        [NOUN]   
4   расстроенный         [ADJ]   
5  

### Sentence not containing

In [ ]:
# load the dataset
with open('/content/sentence_not_containing.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset

random.seed(42)
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample

results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence contains the word
    score = 0 if word in generated_sentence else 1

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)


# to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to JSON files
df.to_csv('eval_sentence_not_containing.csv', index=False)

   example_number                                              input  \
0               1  Составьте предложение, не используя слово "уех...   
1               2   Напишите предложение без слова "беспокойство":\n   
2               3        Напишите предложение без слова "шампунь":\n   
3               4  Придумайте предложение, в котором не будет сло...   
4               5  Придумайте предложение, в котором не будет сло...   
5               6           Напишите предложение без слова "ёлка":\n   
6               7          Напишите предложение без слова "глина":\n   
7               8           Напишите предложение без слова "холм":\n   
8               9  Составьте предложение, не используя слово "зап...   
9              10  Составьте предложение, не используя слово "зая...   

           word word_pos_tags  \
0        уехать        [VERB]   
1  беспокойство        [NOUN]   
2       шампунь        [NOUN]   
3     позволять        [VERB]   
4    бессонница        [NOUN]   
5        

### sentence ends with word

In [ ]:
# to generate a sentence that ends with the given word

def generate_sentence(prompt, max_tokens=50):

    payload = {
        "inputs": prompt,
        "parameters": {"max_tokens": max_tokens}  # add token limit parameter
    }
    response = query(payload)
    if isinstance(response, dict) and "error" in response:
        print(f"Error from API: {response['error']}")
        return ""
    generated_text = response[0]["generated_text"]

    # remove the prompt part from the generated text
    if prompt in generated_text:
        generated_text = generated_text.replace(prompt, "").strip()
    return generated_text

# load the dataset

with open('/content/ends_with_word.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset
random.seed(42)
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample
results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence ends with the word
    score = 1 if generated_sentence.endswith(word) else 0

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)


# to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to JSON files
df.to_csv('eval_ends_with_word.csv', index=False)

    example_number                                              input  \
0                1  Напишите предложение, заканчивающееся словом "...   
1                2  Напишите предложение, которое оканчивается на ...   
2                3  Напишите предложение, которое оканчивается на ...   
3                4  Напишите предложение, последнее слово которого...   
4                5  Напишите предложение, последнее слово которого...   
5                6  Напишите предложение, которое оканчивается на ...   
6                7  Напишите предложение, которое оканчивается на ...   
7                8  Напишите предложение, которое оканчивается на ...   
8                9  Напишите предложение, заканчивающееся словом "...   
9               10  Напишите предложение, заканчивающееся словом "...   
10              11  Напишите предложение, которое оканчивается на ...   
11              12  Напишите предложение, заканчивающееся словом "...   
12              13  Напишите предложение, последнее

## Sentence starts with word

In [ ]:
# load the dataset

with open('/content/starts_with_word.json', 'r', encoding='utf-8') as file:
    dataset = json.load(file)

# select a random sample of 50 examples from the dataset
random.seed(42)
sample_keys = random.sample(list(dataset['examples'].keys()), 50)

# evaluate the random sample
results = []
for idx, key in enumerate(sample_keys):
    item = dataset['examples'][key]
    prompt = item['input']
    word = item['metadata']['word']
    pos_tags = item['metadata']['word_pos_tags']
    template_id = item['metadata']['template_id']

    # to generate the sentence
    generated_sentence = generate_sentence(prompt)

    # check if the generated sentence starts with the word
    score = 1 if generated_sentence.startswith(word) else 0

    # prepare the result
    result = {
        "example_number": idx + 1,
        "input": prompt,
        "word": word,
        "word_pos_tags": pos_tags,
        "generated_sentence": generated_sentence,
        "template_id": template_id,
        "score": score
    }

    results.append(result)

# to evaluate the model's output with expected data

def evaluate_output(results):
    total_score = sum([result["score"] for result in results])
    return total_score

# get scores
total_score = evaluate_output(results)

# create a DataFrame
df = pd.DataFrame(results)

# display the output
print(df[:10])

# print summary of the evaluation
print(f"\nTotal Score: {total_score}")

# save the DataFrame and evaluation scores to JSON files
df.to_csv('eval_starts_with_word.csv', index=False)

    example_number                                              input  \
0                1  Напишите предложение, начинающееся на слово "п...   
1                2  Напишите предложение, которое начинается на сл...   
2                3  Напишите предложение, которое начинается на сл...   
3                4  Напишите предложение, первое слово которого - ...   
4                5  Напишите предложение, первое слово которого - ...   
5                6  Напишите предложение, которое начинается на сл...   
6                7  Напишите предложение, которое начинается на сл...   
7                8  Напишите предложение, которое начинается на сл...   
8                9  Напишите предложение, начинающееся на слово "р...   
9               10  Напишите предложение, начинающееся на слово "в...   
10              11  Напишите предложение, которое начинается на сл...   
11              12  Напишите предложение, начинающееся на слово "в...   
12              13  Напишите предложение, первое сл